In [11]:
import tensorflow as tf
from lib.utils import model ,preproc, newBuild, newGan, sampling , results
from keras.preprocessing.image import ImageDataGenerator
from IPython import display
import os

In [12]:
res = 256
channel = 3
# Preprocessing data with custom function
batch_size = 4
img_shape = (res,res,channel)
epochs = 300
latent_dim = 150
n_images = 16
image_dir = "../oDig/mixed_port_256/"
output_dir = "../oDig/mixed_port_256/output/"
checkpoint_dir = "../checkpoints/mixed_port_256_check/"
data_loc = "../../datasets/mixed_port/"
#data_loc = "../../datasets/celeb/"
image_freq = 100
s = tf.random.normal([n_images, latent_dim]),
checkpoint_freq = 100

In [14]:
constants = dict(
    num_conv_layers=4,
    img_shape=img_shape,
    dim=16,
    output=channel,
    latent_dim=latent_dim,
    kernel=4,
    stride=2
)
callback = dict(
    num_img=n_images,
    cFreq=checkpoint_freq,
    iFreq=image_freq,
    seed=s,
    loc=image_dir
)
gan_attributes = dict(
    check_location=checkpoint_dir,
    b_size=batch_size,
    l_dim=latent_dim,
    epochs=epochs,
)
name = 'Model_1-Same-Tanh-Batch-mDrop'
parameters = dict(
    model_name=name,
    filter_mode=3,
    gen_filter=7,
    disc_filter=6,
    is_batchnorm=True,
    is_multi_drop=True,
    is_tanh=True
)

In [15]:
model1 = model.ModelSettings(**constants, **parameters)

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150)]             0         
                                                                 
 dense_2 (Dense)             (None, 32768)             4947968   
                                                                 
 batch_normalization_13 (Bat  (None, 32768)            131072    
 chNormalization)                                                
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 32768)             0         
                                                                 
 reshape_1 (Reshape)         (None, 16, 16, 128)       0         
                                                                 
 conv2d_transpose_5 (Conv2DT  (None, 32, 32, 128)      262272    
 ranspose)                                               

In [16]:
gen = model.build_generator(model1)

In [17]:
dir = f"{name}"
parent = checkpoint_dir
check_dir = os.path.join(parent,dir)

In [18]:
checkpoint = tf.train.Checkpoint(generator=gen)
ckpt_manager = tf.train.CheckpointManager(checkpoint, check_dir, max_to_keep=5)
if ckpt_manager.latest_checkpoint:
    checkpoint.restore(ckpt_manager.latest_checkpoint).expect_partial()
    print('Checkpoint restored')

In [19]:
 for i in range(100):
    latent_p = results.generate_sample(150, 25)
    X = gen.predict(latent_p)
    display.clear_output(wait=True)
    sampling.plot_results(X, output_dir, i)